<a href="https://colab.research.google.com/github/Mostafa-Hamidifard/cvx-optimization/blob/main/cvx_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import cvxpy as cvx
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=200)
np.random.seed(0x364A_23F2)
N = 500
n = 10
X = np.random.uniform(low=-1, high=1, size=(N, n))
y = np.zeros(N)

_c = np.random.randn(n)
_d = np.random.randn(n)

#We generate data as gaussians with mean and variance dependent on the feature x.
for i in range(N):
    y[i] = np.random.normal(loc=_c.T @ X[i], scale=abs(_d.T @ X[i]))

In [11]:
def phi(j,u):
    return 1/2*cvx.abs(u) + (1/2 - 0.1*j)*u

v = cvx.Variable((9,1))
theta = cvx.Variable((9,n))

cost = 0
for i in range(N):
    for j in range(9):
        xi = cvx.transpose(X[i:i+1,:])
        u = v[j,0] + theta[j:j+1,:] @ xi - y[i]
        cost += phi(j+1,u)
const = []
for j in range(1,9):
    const+= [v[j,0] - v[j-1,0] >= cvx.norm1(theta[j,:]-theta[j-1,:])]

problem = cvx.Problem(cvx.Minimize(cost),const)
problem.solve()


/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:155: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


3129.3211024692932

In [12]:
vopt = v.value
print(vopt)

[[-2.64639137]
 [-1.34229561]
 [-0.69946972]
 [-0.30496747]
 [-0.02012715]
 [ 0.29587617]
 [ 0.86645247]
 [ 1.48435998]
 [ 2.68792298]]


In [13]:
thetaopt = theta.value
print(thetaopt[0,:])

[ 0.34282921 -1.96368959 -1.20296763  0.06182749  1.14701681 -0.20182779 -0.08099914  1.47574762  0.26975849  1.21092238]


In [14]:
Q = vopt.T + X @ thetaopt.T
for i in range(9):
    qi = Q[:,i]
    res = qi - y
    print(np.sum(res >= 0)/500)

0.1
0.194
0.296
0.402
0.502
0.598
0.702
0.806
0.904


fitting model without constraints

In [15]:
problem = cvx.Problem(cvx.Minimize(cost),[])
problem.solve()

3122.429475196474

In [16]:
vopt = v.value
print(vopt)

[[-2.54413669]
 [-1.34728588]
 [-0.74604442]
 [-0.27686327]
 [-0.03760847]
 [ 0.25808255]
 [ 0.88644751]
 [ 1.48845713]
 [ 2.71852354]]


In [17]:
thetaopt = theta.value
print(thetaopt[0,:])

[ 0.53330889 -1.77468807 -1.31942949  0.1528987   1.44823285 -0.13765922 -0.17844532  1.54278969  0.09559465  1.06927663]


In [27]:
x = cvx.Variable((n,1))
const = [cvx.norm_inf(x) <= 1]
j=1
const+= [vopt[j,0] - vopt[j-1,0] + 0.01 <= (thetaopt[j:j+1,:]-thetaopt[j-1:j,:]) @ x]

feasibility_problem = cvx.Problem(cvx.Minimize(0),const)
feasibility_problem.solve()

0.0

In [30]:
xopt = x.value

In [34]:
np.linalg.norm(xopt,np.inf)

0.5993474465338726

In [37]:
vopt[j,0] - vopt[j-1,0] < (thetaopt[j:j+1,:]-thetaopt[j-1:j,:]) @ xopt

array([[ True]])